# Get Started with Batch API

In this notebook, you will learn how to:

*  Submit a batch processing request to an existing service;
*  Check the status and retrieve results of a batch request;

Important: The endpoint_id must be compatible with the model type being used. For example, when using the transcription service, if you select endpoint_id audio-turbo, you must use model whisper-v3-turbo. Mismatched configurations will result in processing errors.



## Install dependencies

In [ ]:
!pip3 install requests

## 1. Prepare audio Sample

In this example, we will download a pre-recorded audio file and then use it to test the batch inference API with transcription requests.

In [ ]:
!curl -L -o "audio.flac" "https://tinyurl.com/4997djsh"

## 2. Set up API credentials

To use the Fireworks Batch API, you'll need your API key. For security reasons, we'll get it from environment variables.

You can set your API key in the notebook by running:

In [ ]:
import os
os.environ["FIREWORKS_API_KEY"] = "your-api-key-here"

## 3. Submitting a Batch Processing Request

This section demonstrates how to submit a new request to the Batch API.

When constructing your request, you'll need to define several parameters. Here are some key ones:

*   **`resource_path`**: Specifies the target API operation. Common paths include:
    *   `"v1/audio/transcriptions"`
    *   `"v1/audio/translations"`
*   **`endpoint_id`**: Identifies the processing endpoint to use (e.g., `"audio-prod"`, `"audio-turbo"`). Ensure this matches the model type if applicable.
*   **`data`**: Contains the configuration specific to the chosen `resource_path`. The required fields and structure are the same as the corresponding synchronous API endpoint.



The following example demonstrates how to send a batch processing request for the transcription service.

In [ ]:
import os
import requests


# API endpoint configuration
API_KEY = os.environ.get("FIREWORKS_API_KEY")
BATCH_URL = "https://batch.us-nevada-1.direct.fireworks.ai/"
# the following setting can be changed
AUDIO_FILE_PATH = "audio.flac"
resource_path = "v1/audio/transcriptions"
endpoint_id = "audio-prod"
data = {"model": "whisper-v3",
          "response_format": "verbose_json",
          "timestamp_granularities": "word",
          "diarize": "true"
          }

URL = BATCH_URL + resource_path
PARAMS = {"endpoint_id": endpoint_id}

response_batch_id = None

try:
    # Check if audio file exists
    if not os.path.exists(AUDIO_FILE_PATH):
        print(f"Error: Audio file {AUDIO_FILE_PATH} not found.")
    else:
        # Prepare the request
        files = {
            "file": open(AUDIO_FILE_PATH, "rb"),
        }
        headers = {
            "Authorization": API_KEY,
        }
    
        # Send POST request
        response = requests.post(URL, files=files, data=data, headers=headers, params=PARAMS)
        
        # Check if the request was successful
        if response.status_code == 200:
            result = response.json()
            response_batch_id = result.get("batch_id")
            print(f"Batch request successful. Batch ID: {response_batch_id}")
        else:
            print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"Error: {str(e)}")


## 3. Check Batch Request Status or Results

To check the status or retrieve the results of a batch request, you'll need to query the API. Due to confidentiality, each query requires your specific `account_id` and the `batch_id` of the request you want to check.
*   `account_id`: Find this on your Fireworks AI account homepage: https://fireworks.ai/account/home
*   `batch_id`: This ID was provided in the response when you initially submitted the batch request.

In [ ]:
import requests

account_id = "" # input your account_id here
batch_id = response_batch_id # or you can input your own batch_id here
url = f"https://batch.us-nevada-1.direct.fireworks.ai/v1/accounts/{account_id}/batch_job/{batch_id}"
headers = {"Authorization": API_KEY}
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    print(response.json())  # Pretty-print JSON response
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

## Conclusion

In this notebook, you learned how to use the Fireworks Batch API to process requests asynchronously. You've seen how to submit batch jobs, check their status, and retrieve results when processing is complete.

Explore the community or reach out to us in [discord](https://discord.gg/fireworks-ai).